In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./kaggle/input\titanic\gender_submission.csv
./kaggle/input\titanic\test.csv
./kaggle/input\titanic\train.csv


In [56]:
import re
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, Trials
from xgboost import XGBClassifier

In [57]:
#加载数据并对数据进行初步分析
train_data = pd.read_csv("./kaggle/input/titanic/train.csv")
test_data = pd.read_csv("./kaggle/input/titanic/test.csv")
#合并训练、测试集数据
alldata = pd.concat([train_data, test_data], axis = 0, ignore_index = True)

In [58]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [59]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [60]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


In [61]:
print(f"训练集有{len(train_data)}个样本数据，测试集有{len(test_data)}样本数据")
print(f"总共有{len(alldata)}个样本数据")

训练集有891个样本数据，测试集有418样本数据
总共有1309个样本数据


In [62]:
#对样本数值数据进行更详细观察，得出其分布特征
round(train_data.describe(percentiles=[.5, .6, .7, .8,.9,]),2)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.00,891.00,891.00,714.00,891.00,891.00,891.00
mean,446.00,0.38,2.31,29.70,0.52,0.38,32.20
std,257.35,0.49,0.84,14.53,1.10,0.81,49.69
min,1.00,0.00,1.00,0.42,0.00,0.00,0.00
50%,446.00,0.00,3.00,28.00,0.00,0.00,14.45
60%,535.00,0.00,3.00,31.80,0.00,0.00,21.68
70%,624.00,1.00,3.00,36.00,1.00,0.00,27.00
80%,713.00,1.00,3.00,41.00,1.00,1.00,39.69
90%,802.00,1.00,3.00,50.00,1.00,2.00,77.96
max,891.00,1.00,3.00,80.00,8.00,6.00,512.33


In [63]:
#对样本类别数据进行更详细观察，得出其分布特征，O即object
train_data.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


In [64]:
#对一些类别特征进行进一步分析，观察其与是否生存之间的相关程度
train_data[['Pclass','Survived']].groupby(['Pclass']).mean().sort_values(by='Survived', ascending=False)

,Survived
Pclass,
1,0.629630
2,0.472826
3,0.242363


In [65]:
train_data[['Sex','Survived']].groupby(['Sex']).mean().sort_values(by='Survived', ascending=False)

,Survived
Sex,
female,0.742038
male,0.188908


In [66]:
train_data[['SibSp','Survived']].groupby(['SibSp']).mean().sort_values(by='Survived', ascending=False)

,Survived
SibSp,
1,0.535885
2,0.464286
0,0.345395
3,0.250000
4,0.166667
5,0.000000
8,0.000000


In [67]:
train_data[['Parch','Survived']].groupby(['Parch']).mean().sort_values(by='Survived', ascending=False)

,Survived
Parch,
3,0.600000
1,0.550847
2,0.500000
0,0.343658
5,0.200000
4,0.000000
6,0.000000


In [68]:
train_data[['Embarked','Survived']].groupby(['Embarked']).mean().sort_values(by='Survived', ascending=False)

,Survived
Embarked,
C,0.553571
Q,0.389610
S,0.336957


In [69]:
#对一些类别数据使用plotly来进行简单的可视化
fig = px.scatter(train_data, x="Age", y="Fare", color="Survived", 
                 hover_data=['Sex', 'Pclass'],
                 title="Titanic乘客生还分布图")
 
fig.show()

In [70]:
#提取乘客称谓到title中
alldata['title'] = alldata.Name.apply(lambda x: re.search(r',\s(.+?)\.', x).group(1))

#统计title称谓计数
alldata.title.value_counts()

title
Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: count, dtype: int64

In [71]:
fig = px.scatter(alldata, x="title", y="Age", color="Sex",
                 title="年龄、姓氏、性别分布图")
fig.show()

In [72]:
#整合称谓信息
alldata.loc[alldata.title.isin(['Ms', 'Mlle']), 'title'] = 'Miss'
alldata.loc[alldata.title.isin(['Mme']), 'title'] = 'Mrs'
rare = ['Major', 'Lady', 'Sir', 'Don', 'Capt', 'the Countess', 'Jonkheer', 'Dona', 'Dr', 'Rev', 'Col']
alldata.loc[alldata.title.isin(rare), 'title'] = 'rare'
alldata.title.value_counts()

title
Mr        757
Miss      264
Mrs       198
Master     61
rare       29
Name: count, dtype: int64

In [73]:
#删除Name数据
alldata.drop(['Name'], axis = 1, inplace = True)

In [74]:
#提取新特征
alldata['family_size'] = alldata.SibSp + alldata.Parch + 1
alldata['ticket_group_count'] = alldata.groupby('Ticket')['Ticket'].transform('count')
alldata['group_size'] = alldata[['family_size', 'ticket_group_count']].max(axis = 1)
alldata['is_alone'] = alldata.group_size.apply(lambda x: 1 if x == 1 else 0)

In [75]:
fig = px.scatter(alldata, x="Pclass", y="Fare", color="Pclass",
                 title="票价，等级分布图")
fig.show()

In [76]:
#提取新特征，并填充缺失值
alldata['fare_p'] = alldata.Fare / alldata.ticket_group_count
alldata.loc[alldata[alldata.fare_p.isna()].index, 'fare_p'] = alldata.groupby('Pclass')['fare_p'].median()[3]

In [77]:
fig = px.scatter(alldata, x="Pclass", y="Fare", color="Pclass",
                 title="票价，等级分布图")
fig.show()

In [78]:
alldata.drop('Fare', axis = 1, inplace = True)
alldata.drop('Ticket', axis = 1, inplace = True)

In [79]:
alldata[alldata.Embarked.isnull()]

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Cabin,Embarked,title,family_size,ticket_group_count,group_size,is_alone,fare_p
61,62,1.0,1,female,38.0,0,0,B28,NaN,Miss,1,2,2,0,40.0
829,830,1.0,1,female,62.0,0,0,B28,NaN,Mrs,1,2,2,0,40.0


In [80]:
alldata.loc[alldata.Embarked.isnull(), 'Embarked'] = 'S'

In [81]:
preprocessing_dummies = pd.get_dummies(alldata[['Pclass', 'Sex', 'Embarked', 'title']],
               columns = ['Pclass', 'Sex', 'Embarked', 'title'],
               prefix = ['pclass', 'sex', 'embarked', 'title'],
               drop_first= False
              )
alldata = pd.concat([alldata, preprocessing_dummies], axis = 1)

In [82]:
alldata.drop(['Pclass', 'Sex', 'Embarked', 'title'], axis = 1, inplace = True)

In [83]:
imputer = KNNImputer(n_neighbors=4)
features = ['SibSp', 'Parch', 'Age',
       'family_size', 'ticket_group_count', 'group_size', 'is_alone',
       'fare_p', 'pclass_1', 'pclass_2', 'pclass_3',
       'sex_female', 'sex_male', 'embarked_C', 'embarked_Q', 'embarked_S',
       'title_Master', 'title_Miss', 'title_Mr', 'title_Mrs', 'title_rare']
all_data_filled = pd.DataFrame(imputer.fit_transform(alldata[features]), columns=features)
alldata['Age'] = all_data_filled['Age']

In [84]:
kmeans = KMeans(n_clusters=4, n_init=10, random_state=41)
labels_pred = kmeans.fit_predict(alldata[['Age']])

In [85]:
# 获取聚类中心，返回每段年龄平均值。
kmeans.cluster_centers_.flatten()

array([36.33665105,  7.0667037 , 54.45      , 23.28408579])

In [86]:
np.argsort(kmeans.cluster_centers_.flatten())

array([1, 3, 0, 2], dtype=int64)

In [87]:
label_dict = {label: v for v, label in enumerate(np.argsort(kmeans.cluster_centers_.flatten()))}
label_dict

{1: 0, 3: 1, 0: 2, 2: 3}

In [88]:
labels = [label_dict[label] for label in labels_pred]
alldata['Age_category'] = labels

In [89]:
fig = px.scatter(alldata, x="Age_category", y="Age", color="Age_category",
                 title="年龄分布图")
fig.show()

In [90]:
alldata.drop(['PassengerId', 'Cabin', 'Age'], axis = 1, inplace = True)

In [91]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    float64
 1   SibSp               1309 non-null   int64  
 2   Parch               1309 non-null   int64  
 3   family_size         1309 non-null   int64  
 4   ticket_group_count  1309 non-null   int64  
 5   group_size          1309 non-null   int64  
 6   is_alone            1309 non-null   int64  
 7   fare_p              1309 non-null   float64
 8   pclass_1            1309 non-null   bool   
 9   pclass_2            1309 non-null   bool   
 10  pclass_3            1309 non-null   bool   
 11  sex_female          1309 non-null   bool   
 12  sex_male            1309 non-null   bool   
 13  embarked_C          1309 non-null   bool   
 14  embarked_Q          1309 non-null   bool   
 15  embarked_S          1309 non-null   bool   
 16  title_

In [92]:
train_clean = alldata.loc[alldata.Survived.notnull()].copy()
test_clean = alldata.loc[alldata.Survived.isnull()].drop('Survived', axis = 1).copy()

In [93]:
X = train_clean.drop('Survived', axis = 1)
y = train_clean.Survived

In [94]:
features = ['family_size', 'group_size', 'is_alone', 'Age_category','ticket_group_count',
       'pclass_1', 'pclass_2', 'pclass_3', 'sex_female', 'sex_male',
       'embarked_C', 'embarked_Q', 'embarked_S', 'title_Master', 'title_Miss',
       'title_Mr', 'title_Mrs', 'title_rare', 'fare_p']
X = X[features].copy()

In [95]:
#划分训练集，验证集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=41) 

In [99]:
def objective(params):
    # 创建 XGBoost 模型，并将 params 作为模型参数
    model = XGBClassifier(**params)
    # 进行交叉验证，计算 AUC
    cv_results = cross_val_score(model, X_train, y_train, cv=10, scoring='roc_auc')
    return cv_results.mean()
 
# 定义搜索空间
space = {
    'n_estimators': hp.randint('n_estimators', 100, 201),
    'max_depth': hp.randint('max_depth', 4, 7),
    'learning_rate': hp.uniform('learning_rate', 0.1, 1),
    'gamma': hp.uniform('gamma', 0, 5),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
}
# 运行贝叶斯优化
trials = Trials()
#最佳参数
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=200, trials=trials)
print(best_params)

100%|██████████| 200/200 [01:42<00:00,  1.95trial/s, best loss: 0.8234677976344642]
{'colsample_bytree': 0.9901095234305655, 'gamma': 0.48397624665135486, 'learning_rate': 0.9300457505183417, 'max_depth': 4, 'min_child_weight': 1.0, 'n_estimators': 179, 'subsample': 0.5121682919306196}


In [100]:
best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9901095234305655, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.48397624665135486,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.9300457505183417,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=1.0, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=179, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [101]:
y_pred = best_model.predict(X_val)  # 使用测试集进行预测
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8268156424581006


In [ ]:
test_pred = best_model.predict(test_clean[features])
pd.DataFrame({
     'PassengerId': test_data.PassengerId,
     'Survived' : test_pred
 }).to_csv('submission.csv', index = False)